In [0]:
dbutils.secrets.listScopes() #checking available scope to fetch key

dbutils.secrets.list("adam-scope") #checking available key inside the scope - keyvault +secret

spark.conf.set("fs.azure.account.key.adamdestorage.dfs.core.windows.net", dbutils.secrets.get(scope="adam-scope", key="adam-key"))  #configure key

In [0]:
## Kimball (Star Schema/Snowflake Schema)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [0]:
# Reading ridership data from Silver
display(dbutils.fs.ls("abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT"))

path,name,size,modificationTime
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/,Kajang/,0,1730237656000
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Putrajaya/,Putrajaya/,0,1730238756000


In [0]:
display(dbutils.fs.ls("abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang"))

path,name,size,modificationTime
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/2019/,2019/,0,1730237702000
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/2020/,2020/,0,1730237686000
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/2021/,2021/,0,1730237682000
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/2022/,2022/,0,1730237671000
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/2023/,2023/,0,1730237657000
abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang/2024/,2024/,0,1730237656000


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DoubleType, DateType
from pyspark.sql.utils import AnalysisException



In [0]:
# ##TO CHECK THE FILE CORRUPTED OR NOT :

# from pyspark.sql import SparkSession
# from pyspark.sql.utils import AnalysisException

# # Initialize Spark session
# spark = SparkSession.builder.appName("CheckParquetValidity").getOrCreate()

# # Path to your Parquet file
# parquet_path = "abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT"

# # Try reading the Parquet file
# try:
#     df = spark.read.parquet(parquet_path)
    
#     # Check if DataFrame is empty
#     if df.count() == 0:
#         print("The Parquet file is empty.")
#     else:
#         print("The Parquet file is valid and contains data.")
#         df.printSchema()  # Print the schema of the DataFrame
#         df.show()         # Show some data from the DataFrame

# except AnalysisException as e:
#     print(f"Failed to read the Parquet file: {e}")


Failed to read the Parquet file: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually. SQLSTATE: 42KD9


In [0]:
# Define the base path (without years and months)
base_path = "abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang"

# List of years/months to read data from
years = ["2019", "2020", "2021", "2022", "2023", "2024"]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# Initialize an empty DataFrame
df = None

# Loop through each year and month to read the Parquet files
# for year in years:
#     for month in months:
#         month_path = f"{base_path}/{year}/{month}"  # Construct path for specific year and month
#         try:
#             # Attempt to read the Parquet files
#             temp_df = spark.read.parquet(month_path)  # Read Parquet files
            
#             # Combine data
#             if df is None:
#                 df = temp_df  # Initialize DataFrame
#             else:
#                 df = df.union(temp_df)  # Union with previous DataFrame
#         except AnalysisException as e:
#             # Ignore the error if the path does not exist
#             if "Path does not exist" in str(e):
#                 print(f"No data for {year} {month}, skipping.")
#             else:
#                 print(f"Failed to read data for {year} {month}: {e}")

# Print the schema of the combined DataFrame if data is found
if df is not None:
    df.printSchema()
else:
    print("No data found for the specified years and months.")


No data for 2024 October, skipping.
No data for 2024 November, skipping.
No data for 2024 December, skipping.
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)



In [0]:
# Loop through each year and print the schema
for year in years:
    year_path = f"{base_path}/{year}/*"  # Path for the specific year
    try:
        temp_df = spark.read.parquet(year_path)  # Read Parquet files
        print(f"Schema for year {year}:")
        temp_df.printSchema()  # Print the schema for each year
    except AnalysisException as e:
        # Ignore the error if the path does not exist
        if "Path does not exist" in str(e):
            print(f"No data found for year {year}, skipping.")
        else:
            print(f"Failed to read data for year {year}: {e}")


Schema for year 2019:
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)

Schema for year 2020:
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)

Schema for year 2021:
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)

Schema for year 2022:
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)

Schema for year 2023:
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)

Schema for year 2024:
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: strin

In [0]:
# Define the schema and base path
schema = StructType([
    StructField("date", DateType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("ridership", DoubleType(), True)
])
base_path = "abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang"

# List of years and months to read data from
years = ["2019", "2020", "2021", "2022", "2023", "2024"]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# Initialize an empty DataFrame
df = None

# Loop through each year and month to read data with specified schema
for year in years:
    for month in months:
        month_path = f"{base_path}/{year}/{month}"  # Construct path
        try:
            temp_df = spark.read.schema(schema).parquet(month_path)  # Read Parquet files
            df = temp_df if df is None else df.union(temp_df)  # Combine data
        except AnalysisException as e:
            if "Path does not exist" in str(e):
                print(f"No data found for {year} {month}, skipping.")
            else:
                print(f"Failed to read data for {year} {month}: {e}")

# Print the schema of the combined DataFrame if data is found
df.printSchema() if df is not None else print("No data found for the specified years and months.")


No data found for 2024 October, skipping.
No data found for 2024 November, skipping.
No data found for 2024 December, skipping.
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- ridership: double (nullable = true)



In [0]:
display(df)

date,year,month,ridership
2019-01-01,2019,January,114173.0
2019-01-02,2019,January,169316.0
2019-01-03,2019,January,175304.0
2019-01-04,2019,January,187891.0
2019-01-05,2019,January,112660.0
2019-01-06,2019,January,95913.0
2019-01-07,2019,January,184365.0
2019-01-08,2019,January,185920.0
2019-01-09,2019,January,188770.0
2019-01-10,2019,January,189818.0


In [0]:
## Create Date Dimension Table

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from datetime import datetime, timedelta

# Initialize Spark session
spark = SparkSession.builder.appName("Date Dimension Table").getOrCreate()

# Define the start and end dates for your date range
start_date = "2019-01-01"  # Adjust to your minimum year
end_date = "2024-12-31"    # Adjust to your maximum year

# Generate a date range as a list of dates
date_list = [(datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=i)).date() for i in range((datetime.strptime(end_date, "%Y-%m-%d") - datetime.strptime(start_date, "%Y-%m-%d")).days + 1)]

# Create a DataFrame from the date list
date_dim = spark.createDataFrame(date_list, DateType()).toDF("date")

# Add year and month columns
date_dim = date_dim.withColumn("year", F.year("date")) \
                   .withColumn("month", F.date_format("date", "MMMM"))

# Write the date dimension table to Azure Data Lake Storage
date_dim.write.mode("overwrite").parquet("abfss://gold@adamdestorage.dfs.core.windows.net/star_schema/fact_table/date_dimension_table")

# Create Fact Table
# fact_table = MRT_Kajang.groupBy("date", "year", "month").agg(F.sum("ridership").alias("total_ridership"))

# # Write the fact table to Azure Data Lake Storage
# fact_table.write.mode("overwrite").parquet("abfss://gold@adamdestorage.dfs.core.windows.net/star_schema/fact_table")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:460)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:577)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:527)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:631)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:651)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, IntegerType, StringType, DoubleType
from pyspark.sql.utils import AnalysisException

# Initialize Spark session
spark = SparkSession.builder.appName("MRT Kajang Analysis").getOrCreate()

# Define the schema and base path
schema = StructType([
    StructField("date", DateType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("ridership", DoubleType(), True)
])
base_path = "abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT/Kajang"

# List of years and months to read data from
years = ["2019", "2020", "2021", "2022", "2023", "2024"]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# Initialize an empty DataFrame
df = None

# Loop through each year and month to read data with specified schema
for year in years:
    for month in months:
        month_path = f"{base_path}/{year}/{month}"  # Construct path
        try:v 
            temp_df = spark.read.schema(schema).parquet(month_path)  # Read Parquet files
            df = temp_df if df is None else df.union(temp_df)  # Combine data
        except AnalysisException as e:
            if "Path does not exist" in str(e):
                print(f"No data found for {year} {month}, skipping.")
            else:
                print(f"Failed to read data for {year} {month}: {e}")

# Print the schema of the combined DataFrame if data is found
if df is not None:
    df.printSchema()

    # Read the date dimension table
    date_dim = spark.read.parquet("abfss://gold@adamdestorage.dfs.core.windows.net/star_schema/fact_table/date_dimension_table")

    # Join the MRT data with the date dimension table on the date field
    joined_df = df.join(date_dim, ["date"], "inner")

    # Calculate average ridership by year and month from the joined data
    average_ridership = joined_df.groupBy(date_dim.year.alias("year"), date_dim.month.alias("month")) \
                                  .agg(F.avg("ridership").alias("average_ridership"))

    # Write the average ridership data to the fact table in Azure Data Lake Storage
    average_ridership.write.mode("append").parquet("abfss://gold@adamdestorage.dfs.core.windows.net/star_scheme/fact_table")

else:
    print("No data found for the specified years and months.")

# Stop the Spark session
spark.stop()


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:460)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:577)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:527)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:631)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:651)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
display(date_dim)

date,year,month
2019-01-01,2019,January
2019-01-02,2019,January
2019-01-03,2019,January
2019-01-04,2019,January
2019-01-05,2019,January
2019-01-06,2019,January
2019-01-07,2019,January
2019-01-08,2019,January
2019-01-09,2019,January
2019-01-10,2019,January


In [0]:
# Operator Dimension
operator_dim = MRT_Kajang.select("operator").distinct()

# Write the operator dimension table to Parquet format
operator_dim.write.mode("overwrite").parquet("abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/operator_dim")



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3010434628395688>, line 2
      1 # Operator Dimension
----> 2 operator_dim = MRT_Kajang.select("operator").distinct()
      4 # Write the operator dimension table to Parquet format
      5 operator_dim.write.mode("overwrite").parquet("abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/operator_dim")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3853, in DataFrame.select(self, *cols)
   3808 def select(self, *cols: "ColumnOrNa

In [0]:
# Create Station Dimension Table
station_dim = spark.createDataFrame([(1, "Kajang"), (2, "KL Sentral"), ...], ["station_id", "station_name"])
station_dim.write.mode("overwrite").parquet("abfss://gold@adamdestorage.dfs.core.windows.net/star_schema/fact_table/station_dimension")


In [0]:
# ETL Process:

# Read LRT and MRT data from the silver layer
lrt_data = spark.read.parquet("abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/LRT")
mrt_data = spark.read.parquet("abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/MRT")

# Combine LRT and MRT data
combined_data = lrt_data.union(mrt_data)

## Load Data into Gold Layer
# Define the output path for the gold layer
gold_layer_path = "abfss://silver@adamdestorage.dfs.core.windows.net/public_transport_data/gold/aggregated_ridership_data"

# Write the combined data to the gold layer
combined_data.write.mode("overwrite").parquet(gold_layer_path)

### Testing and Validation

# Read back the data from the gold layer
gold_data = spark.read.parquet(gold_layer_path)

# Check for consistency in totals
# For example, checking total users for a specific station and year
test_station = "Station_A"  # Replace with an actual station name
test_year = 2023  # Replace with an actual year

total_users_gold = gold_data.filter((F.col("station") == test_station) & (F.col("year") == test_year)).select("total_users").first()[0]

# Compare with the silver layer data
silver_total_users = combined_data.filter((F.col("station") == test_station) & (F.col("year") == test_year)).agg(F.sum("ridership")).first()[0]

if total_users_gold == silver_total_users:
    print("Validation successful: Total users match.")
else:
    print("Validation failed: Total users do not match.")
